In [ ]:
import pickle
import numpy as np
import glob
import os
from tqdm import tqdm

In [ ]:
test_path = os.path.join(".", 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]
len(test_file_list)

In [ ]:
c

In [ ]:
sigmoids.shape

In [ ]:
import random
from PIL import Image
for i in range(10):
    A=sigmoids[random.randint(1,18000)]*255
    im = Image.fromarray(A)
    im.show()

In [ ]:
from tqdm import tqdm
sigmoids = sigmoids[:, y_min_pad:128 - y_max_pad, x_min_pad:128 - x_max_pad]

for i in tqdm(range(len(test_file_list))):
    A=sigmoids[i]*255
    im = Image.fromarray(A).convert('RGB')
    im.save("0859_preds/"+test_file_list[i]+"_pred.png")

In [ ]:
train_path = os.path.join(".", 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]
train_file_list

In [ ]:
sigmoids = pickle.load( open( "train.p", "rb" ) )
from tqdm import tqdm
for i in tqdm(range(len(train_file_list))):
    A=sigmoids[i]*255
    im = Image.fromarray(A).convert('RGB')
    im.save("0859_train/"+train_file_list[i]+"_pred.png")

In [ ]:
sigmoids.shape

In [ ]:
def load_image(path, mask = False):
    """
    Load image from a given path and pad it on the sides, so that eash side is divisible by 32 (newtwork requirement)

    if pad = True:
        returns image as numpy.array, tuple with padding in pixels as(x_min_pad, y_min_pad, x_max_pad, y_max_pad)
    else:
        returns image as numpy.array
    """
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #height, width, _ = img.shape
    height, width = img.shape

    # Padding in needed for UNet models because they need image size to be divisible by 32
    if height % 32 == 0:
        y_min_pad = 0
        y_max_pad = 0
    else:
        y_pad = 32 - height % 32
        y_min_pad = int(y_pad / 2)
        y_max_pad = y_pad - y_min_pad

    if width % 32 == 0:
        x_min_pad = 0
        x_max_pad = 0
    else:
        x_pad = 32 - width % 32
        x_min_pad = int(x_pad / 2)
        x_max_pad = x_pad - x_min_pad

    img = cv2.copyMakeBorder(img, y_min_pad, y_max_pad, x_min_pad, x_max_pad, cv2.BORDER_REFLECT_101)
    if mask:
        # Convert mask to 0 and 1 format
        img = img / 255.0
        return img
    else:
        img = img / 255.0
        return img.reshape((1,128,128))

import cv2
from torch.utils import data
class InferenceDataset(data.Dataset):
    def __init__(self, root_path, file_list, is_test = False):
        self.is_test = is_test
        self.root_path = root_path
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))

        file_id = self.file_list[index]

        image_folder = os.path.join(self.root_path, "images")
        image_path = os.path.join(image_folder, file_id + ".png")

        mask_folder = os.path.join(self.root_path, "masks")
        mask_path = os.path.join(mask_folder, file_id + ".png")

        image = load_image(image_path)

        if self.is_test:
            return file_id
        else:
            mask = load_image(mask_path, mask = True)
            return image, mask

In [ ]:
train_dataset = InferenceDataset(train_path, train_file_list, is_test = True)

In [ ]:
from tqdm import tqdm
file_list=[]
for image in tqdm(data.DataLoader(train_dataset, batch_size = 100)):
    for name in image:
        file_list.append(name)
    

In [ ]:
file_list[0]

In [ ]:
import random
from PIL import Image
sigmoids = pickle.load( open( "train.p", "rb" ) )
A=sigmoids[0]*255
im = Image.fromarray(A)
im.show()


In [ ]:
file_ids = pickle.load( open( "file_ids.p", "rb" ) )

In [ ]:
ids=[]
for a_tuple in file_ids:
    for file in a_tuple:
        ids.append(file)
ids

In [ ]:
import random
from PIL import Image
sigmoids = pickle.load( open( "train.p", "rb" ) )
height, width = 101, 101

if height % 32 == 0:
    y_min_pad = 0
    y_max_pad = 0
else:
    y_pad = 32 - height % 32
    y_min_pad = int(y_pad / 2)
    y_max_pad = y_pad - y_min_pad

if width % 32 == 0:
    x_min_pad = 0
    x_max_pad = 0
else:
    x_pad = 32 - width % 32
    x_min_pad = int(x_pad / 2)
    x_max_pad = x_pad - x_min_pad
sigmoids = sigmoids[:, y_min_pad:128 - y_max_pad, x_min_pad:128 - x_max_pad]
from tqdm import tqdm
for i in tqdm(range(len(ids))):
    A=sigmoids[i]*255
    im = Image.fromarray(A).convert('RGB')
    im.save("0859_train/"+ids[i]+"_pred.png")